In [0]:

print("Hello")

In [0]:

unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-featured/unified/featured.ios14-apps.v1/fact/granularity=daily/year=2021/")
unified_df.limit(10).select("featured_item_visibility_score").show()

In [0]:

from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import *
from pyspark.sql.types import BooleanType

unified = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-featured/unified/featured.ios14-apps.v1/fact/granularity=daily/")
unified.persist(StorageLevel.MEMORY_AND_DISK)
unified.createOrReplaceTempView("unified")

sfOptions = {
  "sfURL" : "appannie_aa_int_prod.us-east-1.snowflakecomputing.com",
  "sfUser" : "app_bdp_data_validation_qa",
  "sfPassword" : "0HN#s@Wa5$1R8jVj",
  "sfDatabase" : "AA_INTELLIGENCE_PRODUCTION",
  "sfSchema" : "ADL_STORE_FREE",
  "sfWarehouse" : "wh_dod_read7"
}
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"
db = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
  .options(**sfOptions) \
  .option("query",  "select * from FACT_FEATURED_IOS14_APPS_V1_CLUSTER_BY_DATE") \
  .load()
db.persist(StorageLevel.MEMORY_AND_DISK)
db.createOrReplaceTempView("db")


In [0]:


_filter = "date='2021-05-01' and country_code='CN' and featured_tab_code='featured-apps'"

b1 = ['country_code', 'device_code', 'featured_category_key', 'featured_us_category_key', 'featured_story_key', 'product_key', 'featured_collection_type_code']
b2 = ['country_code', 'device_code', 'category_item_key', 'featured_category_key', 'featured_story_key', 'product_key', 'collection_type_code']

for i in range(len(b1)):
    sd = spark.sql(f"select {b1[i]}, count(distinct product_key) as cnt1 from unified where {_filter} group by {b1[i]} order by cnt1 desc, {b1[i]} asc limit 10")
    su = spark.sql(f"select {b2[i]} as {b1[i]}, count(distinct product_key) as cnt2 from db where {_filter} group by {b2[i]} order by cnt2 desc, {b1[i]} asc limit 10")
    sd.join(su, [b1[i]], how="full").show()

In [0]:


_filter = "date='2021-05-01' and country_code='CN' and featured_tab_code='featured-apps'"

b1 = ['country_code', 'device_code', 'featured_us_category_key', 'featured_story_key', 'product_key', 'featured_collection_type_code']
b2 = ['country_code', 'device_code', 'featured_category_key', 'featured_story_key', 'product_key', 'collection_type_code']

for i in range(len(b1)):
    sd = spark.sql(f"select {b1[i]}, sum(featured_item_visibility_score) as sum1 from unified where {_filter} group by {b1[i]} order by sum1 desc, {b1[i]} asc limit 10")
    su = spark.sql(f"select {b2[i]} as {b1[i]}, sum(REACH_SCORE) as sum2 from db where {_filter} group by {b2[i]} order by sum2 desc, {b1[i]} asc limit 10")
    sd.join(su, [b1[i]], how="full").show()

In [0]:


_filter = "date='2021-05-01' and country_code='CN' and featured_tab_code='featured-apps'"

for metric in ['reach_score', 'country_code', 'product_key', 'device_code', 'featured_category_key', 'category_item_key', 'collection_label', 'collection_type_code']:
    print(f'{metric}: ', end='')
    spark.sql(f"select count(*) from db where {_filter} and {metric} is NULL limit 10").show(1)


In [0]:


sd = spark.sql(f"select * from unified where date='{_date}'")
su = spark.sql(f"select * from db where date='{_date}'")
joined = sd.join(su, ['country_code', 'device_code', 'featured_story_key', 'product_key'], how="left")
assert sd.count() == joined.count()